# EasySci-RNA Colab Pipeline
This notebook replicates the steps of `EasySci_pipeline.sh` so the analysis can be run directly in a Google Colab environment.

## Clone repository
First clone this repository and change into the project folder.

In [ ]:
!git clone https://github.com/your_user/EasySci.git
%cd EasySci

## Install dependencies
Install the tools required by the pipeline. Versions below match the `README.md` recommendations.

In [ ]:
!apt-get update
!apt-get install -y star samtools parallel
!pip install trim-galore==0.6.10 htseq==2.0.3 pandas==1.5.2 numpy==1.23.5 scipy==1.10.0 biopython==1.81

## Define paths and parameters

In [ ]:
fastq_folder = '/path/to/fastqs'
sample_id_file = '/path/to/sample_ids.txt'
output_folder = '/path/to/output'
star_index = '/path/to/STAR_index'
gtf_file = '/path/to/annotation.gtf'
gtf_file_exons = '/path/to/exon_annotation.gtf'
rt_matching_file = 'script_folder/barcode_files/RT_barcode_matching.txt'
cores = 8
sequencing_type = 'paired-end'  # or 'single-end'

## Rename FASTQ files
Renames R1/R2/R3 files to a standard format.

In [ ]:
import subprocess, pathlib
for sample in open(sample_id_file):
    s = sample.strip()
    if not s:
        continue
    subprocess.run(f'mv {fastq_folder}/*{s}*R1*.fastq.gz {fastq_folder}/{s}.R1.fastq.gz', shell=True)
    subprocess.run(f'mv {fastq_folder}/*{s}*R2*.fastq.gz {fastq_folder}/{s}.R2.fastq.gz', shell=True)
    subprocess.run(f'mv {fastq_folder}/*{s}*R3*.fastq.gz {fastq_folder}/{s}.R3.fastq.gz', shell=True)

## Barcoding reads

In [ ]:
import subprocess, pathlib
script_folder = 'script_folder'
ligation = f'{script_folder}/barcode_files/ligation_barcodes.pickle2'
rt_barcodes = f'{script_folder}/barcode_files/RT_barcodes.pickle2'
randomN = f'{script_folder}/barcode_files/RT_randomN_barcodes.txt'
barcoded = pathlib.Path(output_folder)/'barcoded_fastqs'
barcoded.mkdir(parents=True, exist_ok=True)
if sequencing_type == 'paired-end':
    script = f'{script_folder}/barcoding_reads_paired.py'
    cmd = f'python {script} {fastq_folder} {sample_id_file} {barcoded} {ligation} {rt_barcodes} {cores} {randomN}'
else:
    script = f'{script_folder}/barcoding_reads_single.py'
    cmd = f'python {script} {fastq_folder} {sample_id_file} {barcoded} {ligation} {rt_barcodes} {cores}'
subprocess.run(cmd, shell=True, check=True)

## Trim reads

In [ ]:
import subprocess, pathlib
trimmed = pathlib.Path(output_folder)/'trimmed_fastqs'
trimmed.mkdir(parents=True, exist_ok=True)
commands = []
for sample in open(sample_id_file):
    s = sample.strip()
    if not s: continue
    if sequencing_type == 'paired-end':
        cmd = f'trim_galore --paired {barcoded}/{s}*R1*.gz {barcoded}/{s}*R2*.gz -a2 AAAAAAAA --stringency 3 -o {trimmed}'
    else:
        cmd = f'trim_galore {barcoded}/{s}*R2*.gz -a AAAAAAAA --stringency 3 --three_prime_clip_R1 1 -o {trimmed}'
    commands.append(cmd)
for cmd in commands:
    subprocess.run(cmd, shell=True, check=True)

## STAR alignment

In [ ]:
import subprocess, pathlib
aligned = pathlib.Path(output_folder)/'STAR_alignment'
aligned.mkdir(parents=True, exist_ok=True)
subprocess.run(f'STAR --genomeDir {star_index} --genomeLoad Remove', shell=True)
for sample in open(sample_id_file):
    s = sample.strip()
    if not s: continue
    if sequencing_type == 'paired-end':
        cmd = f'STAR --runThreadN {cores} --outSAMstrandField intronMotif --genomeDir {star_index} --readFilesCommand zcat --readFilesIn {trimmed}/{s}*R1*gz {trimmed}/{s}*R2*gz --outFileNamePrefix {aligned}/{s} --genomeLoad LoadAndKeep'
    else:
        cmd = f'STAR --runThreadN {cores} --outSAMstrandField intronMotif --genomeDir {star_index} --readFilesCommand zcat --readFilesIn {trimmed}/{s}*R2*gz --outFileNamePrefix {aligned}/{s} --genomeLoad LoadAndKeep'
    subprocess.run(cmd, shell=True, check=True)
subprocess.run(f'STAR --genomeDir {star_index} --genomeLoad Remove', shell=True)

## Sort and filter SAM files

In [ ]:
import subprocess, pathlib
filtered = pathlib.Path(output_folder)/'filtered_sam'
filtered.mkdir(parents=True, exist_ok=True)
for sample in open(sample_id_file):
    s = sample.strip()
    if not s: continue
    sam = f'{aligned}/{s}.Aligned.out.sam'
    if sequencing_type == 'paired-end':
        cmd = f'samtools view -q 30 -f 2 -F 780 {sam} | sort -k1,1 -k3,3 -k4,4n > {filtered}/{s}.noheader.sam'
    else:
        cmd = f'samtools view -q 30 -F 772 {sam} | sort -k1,1 -k3,3 -k4,4n > {filtered}/{s}.noheader.sam'
    subprocess.run(cmd, shell=True, check=True)
    subprocess.run(f'grep "@" {sam} > {filtered}/{s}.header.sam', shell=True, check=True)
    subprocess.run(f'cat {filtered}/{s}.header.sam {filtered}/{s}.noheader.sam > {filtered}/{s}.sam', shell=True, check=True)
    subprocess.run(f'rm {filtered}/{s}.header.sam {filtered}/{s}.noheader.sam', shell=True, check=True)

## Remove duplicates

In [ ]:
import subprocess, pathlib
dedup = pathlib.Path(output_folder)/'duplicates_removed'
dedup.mkdir(parents=True, exist_ok=True)
if sequencing_type == 'paired-end':
    dup_script = f'{script_folder}/duplicate_removal_paired.py'
else:
    dup_script = f'{script_folder}/duplicate_removal_single.py'
subprocess.run(f'python {dup_script} {filtered}/ {sample_id_file} {dedup}/ {cores}', shell=True, check=True)

## Calculate read numbers

In [ ]:
import subprocess, pathlib
report = pathlib.Path(output_folder)/'report'
readnum_dir = report/'read_num'
readnum_dir.mkdir(parents=True, exist_ok=True)
with open(readnum_dir/'read_number.csv', 'w') as out:
    out.write('sample,total reads,after filtering barcode,after trimming,uniquely aligned reads,After remove duplicates
')
    for sample in open(sample_id_file):
        s = sample.strip()
        if not s: continue
        total = int(subprocess.check_output(f'zcat {fastq_folder}/{s}*R2*.gz | wc -l', shell=True))//4
        filtered_ct = int(subprocess.check_output(f'zcat {barcoded}/{s}*R2*.gz | wc -l', shell=True))//4
        trim_ct = int(subprocess.check_output(f'zcat {trimmed}/{s}*R2*.gz | wc -l', shell=True))//4
        uniq = int(subprocess.check_output(f'samtools view {filtered}/{s}.sam | wc -l', shell=True))
        dedup_ct = int(subprocess.check_output(f'samtools view {dedup}/{s}.sam | wc -l', shell=True))
        if sequencing_type == 'paired-end':
            uniq//=2
            dedup_ct//=2
        out.write(f'{s},{total},{filtered_ct},{trim_ct},{uniq},{dedup_ct}
')

## Gene counting

In [ ]:
import subprocess, pathlib
gene_dir = report/'Gene_count'
gene_dir.mkdir(parents=True, exist_ok=True)
if sequencing_type == 'paired-end':
    script = f'{script_folder}/gene_counting_paired.py'
else:
    script = f'{script_folder}/gene_counting_single.py'
subprocess.run(f'python {script} {gtf_file} {dedup}/ {gene_dir}/ {sample_id_file} {cores} {randomN}', shell=True, check=True)

## Gene post-processing

In [ ]:
import subprocess, pathlib
gene_out = report/'out/genes'
gene_out.mkdir(parents=True, exist_ok=True)
post_gene = f'{script_folder}/post_processing_genes.py'
subprocess.run(f'python {post_gene} {gene_dir}/ {gene_out}/ {sample_id_file} {rt_matching_file}', shell=True, check=True)

## Exon counting

In [ ]:
import subprocess, pathlib
exon_dir = report/'Exon_count'
exon_dir.mkdir(parents=True, exist_ok=True)
if sequencing_type == 'paired-end':
    exon_script = f'{script_folder}/exon_counting_paired.py'
else:
    exon_script = f'{script_folder}/exon_counting_single.py'
subprocess.run(f'python {exon_script} {gtf_file_exons} {dedup}/ {exon_dir}/ {sample_id_file} {cores}', shell=True, check=True)

## Exon post-processing

In [ ]:
import subprocess, pathlib
exon_out = report/'out/exons'
exon_out.mkdir(parents=True, exist_ok=True)
post_exon = f'{script_folder}/post_processing_exons.py'
subprocess.run(f'python {post_exon} {exon_dir}/ {exon_out}/ {sample_id_file} {rt_matching_file} {sequencing_type}', shell=True, check=True)

## Clean intermediate files

In [ ]:
import shutil
shutil.rmtree(barcoded)
shutil.rmtree(trimmed)
shutil.rmtree(aligned)
shutil.rmtree(filtered)

## Finished

In [ ]:
print('EasySci-RNA pipeline completed')